In [1]:
%%capture
!pip install gradio transformers accelerate numpy
!pip install torch torchvision av hf_xet spaces
!pip install pillow huggingface_hub opencv-python

In [ ]:
from huggingface_hub import notebook_login, HfApi
notebook_login()

In [ ]:
import gradio as gr
from transformers import AutoProcessor, TextIteratorStreamer, AutoModelForImageTextToText
from transformers.image_utils import load_image
from threading import Thread
import time
import torch
import spaces
import cv2
import numpy as np
from PIL import Image

# Helper: progress bar HTML
def progress_bar_html(label: str) -> str:
    return f'''
<div style="display: flex; align-items: center;">
    <span style="margin-right: 10px; font-size: 14px;">{label}</span>
    <div style="width: 110px; height: 5px; background-color: #FFB6C1; border-radius: 2px; overflow: hidden;">
        <div style="width: 100%; height: 100%; background-color: #FF69B4; animation: loading 1.5s linear infinite;"></div>
    </div>
</div>
<style>
@keyframes loading {{
    0% {{ transform: translateX(-100%); }}
    100% {{ transform: translateX(100%); }}
}}
</style>
    '''

# Aya Vision 8B setup
AYA_MODEL_ID = "CohereForAI/aya-vision-8b"
aya_processor = AutoProcessor.from_pretrained(AYA_MODEL_ID)
aya_model = AutoModelForImageTextToText.from_pretrained(
    AYA_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)

def downsample_video(video_path, num_frames=10):
    """
    Extract evenly spaced frames and timestamps from a video file.
    Returns list of (PIL.Image, timestamp_sec).
    """
    vidcap = cv2.VideoCapture(video_path)
    total = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = vidcap.get(cv2.CAP_PROP_FPS) or 30
    indices = np.linspace(0, total-1, num_frames, dtype=int)
    frames = []
    for idx in indices:
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ret, frame = vidcap.read()
        if not ret:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil = Image.fromarray(frame)
        timestamp = round(idx / fps, 2)
        frames.append((pil, timestamp))
    vidcap.release()
    return frames

@spaces.GPU
def process_image(prompt: str, image: Image.Image):
    if image is None:
        yield "Error: Please upload an image."
        return
    if not prompt.strip():
        yield "Error: Please provide a prompt with the image."
        return
    yield progress_bar_html("Processing Image with Aya Vision 8B")
    messages = [{"role": "user", "content": [
        {"type": "image", "image": image},
        {"type": "text", "text": prompt.strip()}
    ]}]
    inputs = aya_processor.apply_chat_template(
        messages, padding=True, add_generation_prompt=True,
        tokenize=True, return_dict=True, return_tensors="pt"
    ).to(aya_model.device)
    streamer = TextIteratorStreamer(aya_processor, skip_prompt=True, skip_special_tokens=True)
    thread = Thread(target=aya_model.generate, kwargs={**inputs, "streamer": streamer, "max_new_tokens": 1024, "do_sample": True, "temperature": 0.3})
    thread.start()
    buff = ""
    for chunk in streamer:
        buff += chunk.replace("<|im_end|>", "")
        time.sleep(0.01)
        yield buff

@spaces.GPU
def process_video(prompt: str, video_file: str):
    if video_file is None:
        yield "Error: Please upload a video."
        return
    if not prompt.strip():
        yield "Error: Please provide a prompt with the video."
        return
    yield progress_bar_html("Processing Video with Aya Vision 8B")
    frames = downsample_video(video_file)
    # Build chat messages with each frame and timestamp
    content = [{"type": "text", "text": prompt.strip()}]
    for img, ts in frames:
        content.append({"type": "text", "text": f"Frame at {ts}s:"})
        content.append({"type": "image", "image": img})
    messages = [{"role": "user", "content": content}]
    inputs = aya_processor.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True,
        return_dict=True, return_tensors="pt"
    ).to(aya_model.device)
    streamer = TextIteratorStreamer(aya_processor, skip_prompt=True, skip_special_tokens=True)
    thread = Thread(target=aya_model.generate, kwargs={**inputs, "streamer": streamer, "max_new_tokens": 1024, "do_sample": True, "temperature": 0.3})
    thread.start()
    buff = ""
    for chunk in streamer:
        buff += chunk.replace("<|im_end|>", "")
        time.sleep(0.01)
        yield buff

# Build Gradio UI
demo = gr.Blocks()
with demo:
    gr.Markdown("# **Aya Vision 8B Multimodal: Image & Video**")
    with gr.Tabs():
        with gr.TabItem("Image Inference"):
            txt_i = gr.Textbox(label="Prompt", placeholder="Enter prompt...")
            img_u = gr.Image(type="filepath", label="Image")
            btn_i = gr.Button("Run Image")
            out_i = gr.Textbox(label="Output", interactive=False)
            btn_i.click(fn=process_image, inputs=[txt_i, img_u], outputs=out_i)
        with gr.TabItem("Video Inference"):
            txt_v = gr.Textbox(label="Prompt", placeholder="Enter prompt...")
            vid_u = gr.Video(label="Video")
            btn_v = gr.Button("Run Video")
            out_v = gr.Textbox(label="Output", interactive=False)
            btn_v.click(fn=process_video, inputs=[txt_v, vid_u], outputs=out_v)

demo.launch(debug=True, share=True)